In [1]:
import tensorflow as tf
import numpy as np
import os
import pickle
from sklearn.model_selection import train_test_split

# unpack the pickle
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
dataset_dir = '/content/drive/MyDrive/FALL_2022_SEMESTER/COMS_4995_Applied_Deep_Learning/Course_Project/music_dataset/'
pickle_file_path = os.path.join(dataset_dir, 'spec_band_pic.pickle')

In [3]:
# unpack the pickle file
pickle_in = open(pickle_file_path, "rb")
test_data = pickle.load(pickle_in)

In [4]:
# map each raga str to an int
raga_to_int = {
    'Sankarabaranam': 0,
    'Reethigowla': 1,
    'Kalyani': 2,
    'Shanmukhapriya': 3,
    'SindhuBhairavi': 4,
    'Sri': 5,
    'Surutti': 6,
    'Thodi': 7,
    'PoorviKalyani': 8,
    'Mohanam': 9,
    'Madhyamavathi': 10,
    'Kharaharapriya': 11,
    'Keeravani': 12,
    'Kapi': 13,
    'Kambhodhi': 14,
    'Kamas': 15, 
    'Jaganmohini': 16,
    'Hamsadhwani': 17,
    'Bilahari': 18,
    'Bhairavi': 19
}


In [5]:
# OK so now just create a random network and see how it does
# split up into test and train data
X = []
y = []
for val in test_data:
  X.append(val[0])
  y.append(raga_to_int[val[1]])

del test_data




In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                   random_state=104, 
                                   test_size=0.25, 
                                   shuffle=True)
X_train =np.array(X_train)
X_test =np.array(X_test)
y_train =np.array(y_train)
y_test =np.array(y_test)

In [7]:
print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_test shape: {y_test.shape}')

X_train shape: (1173, 1, 2584)
X_test shape: (391, 1, 2584)
y_train shape: (1173,)
y_test shape: (391,)


In [8]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(1, 2584)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(20)
])

In [9]:
predictions = model(X_train[:1]).numpy()
predictions

array([[ 2786.3325 ,  7090.722  , -7575.895  , -4185.679  ,  5592.446  ,
         -651.34814,   422.70987,  3499.9602 , -5874.8833 , -6764.052  ,
         1535.539  ,   737.9678 ,  5281.1655 ,  -895.79584,  7953.2144 ,
          469.03735,  3021.5422 ,  2283.5605 ,  2776.6367 ,  1471.5856 ]],
      dtype=float32)

In [10]:
tf.nn.softmax(predictions).numpy()


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0.]], dtype=float32)

In [11]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


In [12]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [13]:
model.fit(X_train, y_train, epochs=30)


Epoch 1/30
37/37 [==============================] - 1s 5ms/step - loss: 6074.9780 - accuracy: 0.0708
Epoch 2/30
37/37 [==============================] - 0s 5ms/step - loss: 3.1471 - accuracy: 0.1057
Epoch 3/30
37/37 [==============================] - 0s 5ms/step - loss: 2.9663 - accuracy: 0.2097
Epoch 4/30
37/37 [==============================] - 0s 5ms/step - loss: 2.9476 - accuracy: 0.2285
Epoch 5/30
37/37 [==============================] - 0s 5ms/step - loss: 2.9293 - accuracy: 0.2285
Epoch 6/30
37/37 [==============================] - 0s 5ms/step - loss: 2.9115 - accuracy: 0.2285
Epoch 7/30
37/37 [==============================] - 0s 4ms/step - loss: 2.8941 - accuracy: 0.2285
Epoch 8/30
37/37 [==============================] - 0s 5ms/step - loss: 2.8774 - accuracy: 0.2285
Epoch 9/30
37/37 [==============================] - 0s 5ms/step - loss: 2.8614 - accuracy: 0.2285
Epoch 10/30
37/37 [==============================] - 0s 5ms/step - loss: 2.8458 - accuracy: 0.2285
Epoch 11/30
37/3

In [14]:
# might have started to overfit on trarin accuracy after 1 epoch???
# try testing it 
model.evaluate(X_test,  y_test, verbose=2)


13/13 - 0s - loss: 2.6891 - accuracy: 0.2123 - 164ms/epoch - 13ms/step


[2.6891300678253174, 0.21227622032165527]

In [15]:
# ok so now, get the actual features of the dataset (Spectral centroid, spectral bandwidth, mel-spectrogram, Mel freq cepstral coeff, chroma)
# Get those into a pickle file, and see how it performs on the same neural network. If accuracy is low, then either modify network or add more data samples